# Tranforming Data


In [1]:
document_url = 'https://cdnbbsr.s3waas.gov.in/s380537a945c7aaa788ccfcdf1b99b5d8f/uploads/2023/05/2023050195.pdf'


In [2]:
!pip install requests
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 940.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 45.6 MB/s eta 0:00:00


In [ ]:
import requests
import pdfplumber

def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as pdf_file:
        pdf_file.write(response.content)

def extract_text_from_pdf(pdf_path):
  with pdfplumber.open(pdf_path) as pdf:
      text = ""
      for page in pdf.pages:
          text += page.extract_text()

      return text

# Example usage
pdf_url = "https://example.com/path/to/your.pdf"
downloaded_pdf_path = "document.pdf"

download_pdf(document_url, downloaded_pdf_path)
extracted_text = extract_text_from_pdf(downloaded_pdf_path)

In [ ]:
print(extracted_text[:1000])

In [ ]:
import re

def remove_non_alphanumeric(input_string):
    # Use regular expression to remove non-alphanumeric characters
    clean_string = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return clean_string.replace('\n', ' ')

wanted_text = remove_non_alphanumeric(extracted_text)

In [ ]:
wanted_text[:6004]

In [ ]:
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

unique_words = len(set(word_tokenize(wanted_text)))

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([wanted_text])
text_encode = [tokenizer.texts_to_sequences(wanted_text)]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
# Assuming all_text_encd is a list of arrays with varying lengths
# Pad sequences to a fixed length
padded_sequences = pad_sequences(text_encode[0], dtype='int32', padding='post', truncating='post')

# Convert to NumPy array
all_text_encd = np.array(padded_sequences).reshape(1,-1).tolist()[0]

In [ ]:
all_text_encd

In [ ]:
block = 12
total_embd = round(len(all_text_encd)/block)

In [ ]:
total_embd

In [ ]:

x_vectors = []
y_vectors = []

ind = 0
for i in range(total_embd-1):
  for n in range(block):
    seq = ind+n
    x_vectors.append(all_text_encd[ind:seq+1])
    y_vectors.append(all_text_encd[seq+1])
  ind+=12


In [ ]:
len(x_vectors), len(y_vectors)

# Reforming Data

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import convert_to_tensor
import tensorflow as tf
import numpy as np

x = pad_sequences(x_vectors, maxlen=block, padding='pre')
#y = convert_to_tensor(y_vectors, dtype=tf.float32)
y = np.array(y_vectors)

In [ ]:
new_shape = (x.shape[0], x.shape[1], 1)
x = x.reshape(new_shape)

In [ ]:
classes = y.max()+1

In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=classes)
#y = y.reshape(-1,1)

In [ ]:
x.shape, y.shape

# Creating Model

In [ ]:
!pip install torch

In [ ]:
import torch
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
unique_words, block

(7056, 12)

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(input_dim=unique_words, output_dim=150, input_length=block))
#model.add(Dense(12))
model.add(LSTM(256, input_shape=(12, 1)))
model.add(Dense(3698, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 150)           1058400   
                                                                 
 lstm (LSTM)                 (None, 256)               416768    
                                                                 
 dense (Dense)               (None, 3698)              950386    
                                                                 
Total params: 2,425,554
Trainable params: 2,425,554
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x, y, epochs=110)

In [ ]:
p = model.predict(x[34])[0]
p

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

model.save('/content/drive/indian_constitution.h5')